<a href="https://colab.research.google.com/github/ifymifan/IOT-AI/blob/main/LAB_5_1_1_Spark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
!pip install findspark
import findspark
import pyspark # this was also missing in the original code
findspark.init()
findspark.find()
# import findsoark # typo in the original code. I'm assuming the user meant to type findspark?
findspark.find() # if user did in fact mean findsoark, then user should !pip install findsoark and import findsoark

'/usr/local/lib/python3.11/dist-packages/pyspark'

In [84]:
from pyspark.sql import SparkSession

from google.colab import drive
drive.mount('/content/drive')

# Create a spark Session

spark =  SparkSession.builder\
        .master("local")\
        .appName("Titanic data")\
        .getOrCreate()

# Check spark session information
spark # Changed 'Spark' to 'spark' to access the SparkSession object

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
from pyspark.sql import SparkSession

In [86]:
df = (spark.read \
    .format("csv") \
    .option("header", "true") \
    .load("/content/drive/MyDrive/Titanic/train.csv")) # Load using a relative path

In [87]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [88]:
from pyspark.sql.functions import col

In [89]:
dataset = df.select(col('Survived').cast('float'),
                         col('Pclass').cast('float'),
                         col('Sex'),
                         col('Age').cast('float'),
                         col('Fare').cast('float'),
                         col('Embarked')
                        )

In [90]:
dataset.show(4)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
+--------+------+------+----+-------+--------+
only showing top 4 rows



In [91]:
from pyspark.sql.functions import isnull, when, count, col

In [92]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [93]:
dataset = dataset.replace('?',None)\
.dropna(how='any')
#

In [94]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [95]:
dataset.show(3)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
+--------+------+------+----+-------+--------+
only showing top 3 rows



In [98]:
from pyspark.ml.feature import StringIndexer # Changed 'mi' to 'ml' in the import statement

In [99]:
dataset = StringIndexer(
    inputCol='Sex',
    outputCol='Gender',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [31]:
dataset.show(2)

+--------+------+------+----+-------+--------+------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|
+--------+------+------+----+-------+--------+------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|
+--------+------+------+----+-------+--------+------+
only showing top 2 rows



In [100]:
#Drop unnecessary comums
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')


In [101]:
#Assembly all the feautres with vectorAssembly
from pyspark.ml.feature import VectorAssembler
required_features = ['Pclass',
                    'Age',
                    'Fare',
                    'Gender'] # Removed 'Boarder' and kept 'Gender'

# required_features should only contain existing columns: 'Pclass', 'Age', 'Fare', 'Gender', 'Survived'
# 'Boarder' does not exist and was causing the error.
Assembly = VectorAssembler(inputCols=required_features,outputCol='features')
transformed_data = Assembly.transform(dataset)
transformed_data.show(5)

+--------+------+----+-------+------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|            features|
+--------+------+----+-------+------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0| [3.0,22.0,7.25,0.0]|
|     1.0|   1.0|38.0|71.2833|   1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+--------------------+
only showing top 5 rows



In [102]:
# Splitting dataset into train and test
(training_data, test_data) = dataset.randomSplit([0.8,0.2]) # Changed 'transfermed_data' to 'dataset'
print("Number of train samplea: " + str(training_data.count()))
print("Number of test samplea: " + str(test_data.count()))

Number of train samplea: 581
Number of test samplea: 131


In [103]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)

In [125]:
rf = pyspark.ml.classification.RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)


In [136]:
# Splitting dataset into train and test
# (training_data, test_data) = dataset.randomSplit([0.8,0.2]) # Changed 'transfermed_data' to 'dataset'
# Use the transformed_data DataFrame which contains the 'features' column
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])
print("Number of train samplea: " + str(training_data.count()))
print("Number of test samplea: " + str(test_data.count()))

Number of train samplea: 567
Number of test samplea: 145


In [139]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)

In [143]:
Model = rf.fit(training_data)

In [145]:
prediction = Model.transform(test_data)

In [146]:
#Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='Survived',
    predictionCol='prediction',
    metricName='accuracy')

In [150]:
accuracy = evaluator.evaluate(prediction) # Use 'prediction' instead of 'predictions_test'
print('Training Accuracy = ' , accuracy)

Training Accuracy =  0.8
